# Setup


In [1]:
import numba
import numpy as np

Create a small array of random number such that it can be used to test the algorithms to make sure they work.


In [4]:
np.random.seed(42)
test_array = np.random.randint(1, 100_000, 100, dtype=np.int32)
test_array[0:10]

array([15796,   861, 76821, 54887,  6266, 82387, 37195, 87499, 44132,
       60264], dtype=int32)

# Casual Use


## Implementation #1

Implementation edited slightly to take in an optional input array parameter.


In [8]:
import random


def digit_sum(n):
    """Calculate the sum of digits of a number"""
    return sum(int(digit) for digit in str(n))


def find_difference(numbers=None):
    if not numbers:
        # Generate list of 1 million random integers
        numbers = [random.randint(1, 100000) for _ in range(1000000)]

    # Initialize variables for min and max numbers with digit sum 30
    min_num = float("inf")  # Initialize to positive infinity
    max_num = float("-inf")  # Initialize to negative infinity

    # Find numbers whose digits sum to 30
    for num in numbers:
        if digit_sum(num) == 30:
            min_num = min(min_num, num)
            max_num = max(max_num, num)

    # Check if we found any numbers with digit sum 30
    if min_num == float("inf") or max_num == float("-inf"):
        return "No numbers found with digit sum of 30"

    return max_num - min_num


def casual_implementation_1(numbers=None):
    return find_difference(numbers)

In [14]:
%%time

casual_implementation_1(test_array.tolist())

CPU times: user 138 μs, sys: 1 μs, total: 139 μs
Wall time: 142 μs


62910

Benchmark the code: due to slowness, only do 10 trials for this implementation.


In [15]:
%%timeit -n 1 -r 10

casual_implementation_1()

632 ms ± 16.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


## Implementation #2

Implementation edited slightly to take in an optional input array parameter.

For a fair benchmark, the `DigitSumFinder` is pre-instantiated.


In [22]:
import random
from array import array
from typing import Tuple, Optional
import time


class DigitSumFinder:
    def __init__(
        self,
        target_sum: int = 30,
        range_start: int = 1,
        range_end: int = 100_000,
        count: int = 1_000_000,
    ):
        self.target_sum = target_sum
        self.range_start = range_start
        self.range_end = range_end
        self.count = count

        # Pre-calculate digit sums for all possible numbers
        self.digit_sums = self._precompute_digit_sums()

    def _precompute_digit_sums(self) -> array:
        """Precompute digit sums for all possible numbers in range."""
        digit_sums = array("B", [0] * (self.range_end + 1))
        for num in range(self.range_start, self.range_end + 1):
            total = 0
            n = num
            while n:
                total += n % 10
                n //= 10
            digit_sums[num] = total
        return digit_sums

    def find_difference(
        self, input_numbers=None
    ) -> Tuple[int, Optional[int], Optional[int]]:
        """
        Find the difference between max and min numbers with target digit sum.
        Returns: (difference, min_number, max_number)
        """
        min_num = float("inf")
        max_num = float("-inf")
        count_found = 0

        if input_numbers:
            for num in input_numbers:
                if self.digit_sums[num] == self.target_sum:
                    count_found += 1
                    if num < min_num:
                        min_num = num
                    if num > max_num:
                        max_num = num

        else:
            # Generate and process random numbers
            for _ in range(self.count):
                num = random.randint(self.range_start, self.range_end)
                if self.digit_sums[num] == self.target_sum:
                    count_found += 1
                    if num < min_num:
                        min_num = num
                    if num > max_num:
                        max_num = num

        if count_found == 0:
            return 0, None, None

        return max_num - min_num, min_num, max_num


def casual_implementation_2(d, numbers=None):
    return d.find_difference(numbers)


d = DigitSumFinder()

In [24]:
%%time

casual_implementation_2(d, test_array.tolist())

CPU times: user 26 μs, sys: 0 ns, total: 26 μs
Wall time: 28.8 μs


(62910, 23898, 86808)

In [30]:
%%timeit -n 1 -r 10

casual_implementation_2(d)

237 ms ± 8.02 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
